In [50]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_1.csv')
df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [51]:
# 결측치가 하나라도 존재하는 행의 경우 경우 해당 행을 삭제하라. 
#그후 남은 데이터의 상위 70%에 해당하는 데이터만 남겨둔 후
#median_income 컬럼의 1분위수를 반올림하여 소숫점이하 2째자리까지 구하여라

In [52]:
df.isnull().sum()  # housing_median_age , total_bedrooms , population ,median_income, median_house_value

longitude               0
latitude                0
housing_median_age    323
total_rooms             0
total_bedrooms        207
population            950
households              0
median_income         234
median_house_value     44
ocean_proximity         0
dtype: int64

In [55]:
df = df.dropna()
df_result = df.iloc[:int(len(df)* 0.7)]
df_result = df_result['median_income'].quantile(0.25).round(2)
print(df_result)

2.51


In [56]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_2.csv')
df.head()

,Country Name,Country Code,Year,Value
0,Arab World,ARB,1968,2576068.3
1,Arab World,ARB,1969,2843420.4
2,Arab World,ARB,1970,3138550.0
3,Arab World,ARB,1971,3642691.0
4,Arab World,ARB,1972,4331605.7


In [57]:
# 1990년도는 해당년도 평균 이하 GDP를 가지지만, 
#2010년도에는 해당년도 평균 이상 GDP를 가지는 국가의 숫자를 구하여라

In [76]:
df_1990 = df[df['Year'] == 1990]
df_2010 = df[df['Year'] == 2010]

df_1900_result = df_1990[df_1990['Value'] <= df_1990['Value'].mean()]
df_2010_result = df_2010[df_2010['Value'] >= df_2010['Value'].mean()]

print(set(df_1900_result['Country Code']) & set(df_2010_result['Country Code']))

{'TEA', 'MEA', 'CHN', 'EAP', 'BRA'}


In [77]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_3.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,2,1,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3,1,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,C85,S
3,4,1,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5,0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,C123,S


In [78]:
# 데이터에서 결측치가 가장 많은 컬럼을 출력하라

In [80]:
df.isnull().sum()  # Fare

PassengerId     0
Survived        0
Pclass         55
Name           61
Sex            64
Age            44
SibSp          55
Parch          53
Ticket         34
Fare           69
Cabin          58
Embarked       49
dtype: int64

In [119]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_test_.csv')
train.head()

# print(train.columns)
# print(test.columns)

# train.isnull().sum()
# test.isnull().sum()

X_train = train.drop(["TravelInsurance"], axis = 1)
X_test = test.drop(["ID"], axis =1)
Y_train = train['TravelInsurance']


X_train = pd.get_dummies(X_train)
X_test=  pd.get_dummies(X_test)

X_test = X_test.reindex(columns = X_train.columns, fill_value = 0)


from sklearn.model_selection import train_test_split
X_train, X_val, Y_train , Y_val = train_test_split(X_train, Y_train, test_size = 0.2)


from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train,Y_train)
y_pred_proba = model.predict_proba(X_val)
# print(y_pred_proba)

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(Y_val,y_pred_proba[:,1])
print(roc_auc)

y_pred = model.predict_proba(X_test)
# print(y_pred)
result = pd.DataFrame(y_pred[:,1] , columns=["proba"])
result_sum = pd.concat([test["ID"], result], axis = 1)  # concat 합치기 
result_sum.to_csv("result.csv",index =False)


result = pd.read_csv("result.csv")
print(result)

0.8020807156857818
       ID  proba
0       6   0.31
1       9   0.32
2      20   0.29
3      21   0.26
4      23   0.27
..    ...    ...
492  1964   0.36
493  1970   0.26
494  1973   0.97
495  1976   0.63
496  1982   0.98

[497 rows x 2 columns]


In [1]:
import pandas as pd 
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p3_1.csv')
df.head()

,ID,투약전,투약후
0,user_1,55.137,56.106
1,user_2,66.584,60.409
2,user_3,52.259,50.639
3,user_4,77.081,69.164
4,user_5,62.677,57.622


In [2]:
#  투약 후 체중에서 투약 전 체중을 뺏을 때 값 
# -3이하 : A등급, -3초과  0이하 : B등급, 0 초과 : C등급. 약 실험에서 A,B,C 그룹간의 인원 수 비율은 2:1:1로 알려져 있다. 

In [3]:
df['diff'] = df['투약후'] - df['투약전']

In [4]:
import pandas as pd 
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p3_1.csv')
df.head() 

df['diff'] = df['투약후'] - df['투약전']

def g(x):
    if x <= -3:
        return 'A등급'
    if x <= 0:
        return 'B등급'
    if x > 0:
        return 'C등급'

df['등급'] = df['diff'].apply(g)

observed_counts = df['등급'].value_counts().sort_index()
total = observed_counts.sum()
expected_counts = [total * 0.5 , total * 0.25 , total * 0.25]
# print(expected_counts)

from scipy.stats import chisquare

stats , p = chisquare(observed_counts,expected_counts)
print(chisquare(observed_counts,expected_counts))
print(round(stats,3))

Power_divergenceResult(statistic=np.float64(3.6127659574468085), pvalue=np.float64(0.1642471491200832))
3.613


In [240]:
# 카이제곱 검정 p값을 반올림하여 소숫점 이하 3자리까지 구하고, 
#유의수준 0.05하에서 귀무가설과 대립가설중 유의한 가설을 하나를 선택하시오(귀무/대립)

print(round(p,3))

if 0.164 > 0.05:
    print("귀무 가설 채택")
else:
    print("귀무 가설 기각")

0.164
귀무 가설 채택


In [241]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p3_2_.csv')
df.head()

,rpm,group
0,163.6,A
1,246.9,A
2,234.9,A
3,156.9,A
4,226.6,A


In [243]:
# A,B 공장 각각 정규성을 가지는지 샤피로 검정을 통해 확인하라. (각 공장의 pvalue 출력할 것)

In [249]:
df_a = df[df['group'] == 'A']['rpm']
df_b = df[df['group'] == 'B']['rpm']

from scipy.stats import shapiro

print(shapiro(df_a))
print(shapiro(df_b))

ShapiroResult(statistic=np.float64(0.9863896921013734), pvalue=np.float64(0.3979080924239819))
ShapiroResult(statistic=np.float64(0.9943944976838186), pvalue=np.float64(0.9562843067714428))


In [250]:
# A,B 공장 생산 제품의 rpm은 각각 정규성을 가지는지  등분산성 확인. (각 공장의 pvalue 출력할 것)

In [252]:
from scipy.stats import levene

print(levene(df_a,df_b))

LeveneResult(statistic=np.float64(0.014634456943777612), pvalue=np.float64(0.9038351280607178))


In [260]:
# 대응 표본 t 검정을 통해 B공장 제품들의 rpm이 A 공장 제품의 rpm보다 크다고 말할 수 있는지 검정하라.
#pvalue를 소숫점 이하 3자리까지 출력하고 귀무가설, 대립가설 중 하나를 출력하라*

# 같은 그룹간의 비교 = > 대응 표본 => ttest_rel =
# b가 a보다 큰가? alternative = 'greater'
# b가 a보다 작은가? alternative = 'less'

In [259]:
from scipy.stats import ttest_rel
stats , p = ttest_rel(df_a,df_b , alternative = 'greater')
print(round(p,3))
if 0.017 > 0.05:
    print("귀무")
else:
    print("대립")

0.991
대립
